In [ ]:
## roadTrip.ipynb -- visiting each city (and pulling all the data)

to make CSV file:
* go to lennar, xpath: /html/body/div[1]/div/main/div/div[2]/footer/div[1]/div[1]/div[3]/div/div[2]/div/div//a/@href

In [1]:
import pandas as pd
import os

In [2]:
cities = pd.read_csv('./cities-10-2023.csv')

In [3]:
cities

,market,state,url
0,coastal-alabama,alabama,https://www.lennar.com/new-homes/alabama/coast...
1,huntsville,alabama,https://www.lennar.com/new-homes/alabama/hunts...
2,phoenix,arizona,https://www.lennar.com/new-homes/arizona/phoenix
3,tucson,arizona,https://www.lennar.com/new-homes/arizona/tucson
4,fresno-bakersfield,california,https://www.lennar.com/new-homes/california/fr...
...,...,...,...
67,berkeley-county,west-virginia,https://www.lennar.com/new-homes/west-virginia...
68,jefferson-county,west-virginia,https://www.lennar.com/new-homes/west-virginia...
69,boise,idaho,https://www.lennar.com/new-homes/idaho/boise
70,madison,wisconsin,https://www.lennar.com/new-homes/wisconsin/mad...


In [4]:
# cities['state'] = cities['url'].str.extract(r'/new-homes/([^/]+)/')
# cities['market'] = cities['url'].str.extract(r'/new-homes/[^/]+/([^/]+)')

In [5]:
# cities['url'] = "https://www.lennar.com"+ cities['url']

In [6]:
from tools import *

In [7]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [8]:
cities['url'][0]

'https://www.lennar.com/new-homes/alabama/coastal-alabama'

In [15]:
s=Service(ChromeDriverManager().install())    
driver = webdriver.Chrome(service=s)
driver.maximize_window()

driver.get(cities['url'][0])
time.sleep(10)

closeCookies(driver)

	 Closed cookies window. Continue...


In [17]:
# driver.find_element(By.XPATH, 
#                             "//*[main]").get_attribute('innerHTML')


In [23]:
main_element = driver.find_element(By.XPATH, 
                            '//*[@id="main"]/div/div[1]/div[2]/div/div[3]/div/div[2]/div[1]')


In [24]:
link_element = main_element.find_element(By.TAG_NAME, 'a')
link_element

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"a"}
  (Session info: chrome=117.0.5938.149)
Stacktrace:
0   chromedriver                        0x0000000100eeed68 chromedriver + 4337000
1   chromedriver                        0x0000000100ee6de4 chromedriver + 4304356
2   chromedriver                        0x0000000100b13a5c chromedriver + 293468
3   chromedriver                        0x0000000100b58d50 chromedriver + 576848
4   chromedriver                        0x0000000100b4e200 chromedriver + 532992
5   chromedriver                        0x0000000100b93908 chromedriver + 817416
6   chromedriver                        0x0000000100b4ca5c chromedriver + 526940
7   chromedriver                        0x0000000100b4d908 chromedriver + 530696
8   chromedriver                        0x0000000100eb4db4 chromedriver + 4099508
9   chromedriver                        0x0000000100eb9270 chromedriver + 4117104
10  chromedriver     

In [ ]:
s=Service(ChromeDriverManager().install())    
driver = webdriver.Chrome(service=s)
driver.maximize_window()

for u in cities['url'][:1]:
    state, city = u.split('/')[-2], u.split('/')[-1]
    print(f"Scraping data for {city}, {state}")

    driver.get(u)
    time.sleep(15)

    closeCookies(driver)

    total = driver.find_element(By.XPATH, '//*[@id="main"]//div[1]/div[2]//div[1]//div[4]/div[1]//div[1]//button[1]').text.splitlines()[0]
    
    # assert 'home' in total
    total = int(total.split(' ')[0])
    print(f"\t Found {total} homes")
    time.sleep(3)
    
    driver.find_element(By.XPATH, '//*[@id="main"]//div[1]/div[2]//div[1]//div[4]/div[1]//div[1]//button[1]').click()
    loadMoreHomes(driver)

    df = scrapeItUp(driver)
    # they don't count future releases in available homes anymore. change to <=
    assert total <= df.shape[0], "Problem scraping. Fuck me"
        # print(f'{total}. Done!', end='')

    df_clean = cleanItUp(df)

    outdir = f'./data-10-2023/{state}'
    if not os.path.exists(outdir):
        os.mkdir(outdir)

    df_clean.to_csv(f'./data-10-2023/{state}/{city}.csv', index = False)
    time.sleep(5)

In [ ]:
# driver.find_element(By.XPATH, f'//*[@id="main"]/div/div[1]/div[2]/div/div[3]/div/div[2]/div[1]').text.splitlines()

In [ ]:
# cities['url']